In [ ]:

import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from pathlib import Path
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, losses, metrics, callbacks
from tensorflow.keras import Sequential, Model, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")
random.seed(45)

print(tf.__version__)
import glob
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load metadata to get age information
HOME_PATH = os.getcwd() + "/"
meta_train = pd.read_csv(HOME_PATH + 'meta_train.csv')

# Create mapping from image_id to age
image_to_age = dict(zip(meta_train['image_id'], meta_train['age']))

# Load images with age labels
def load_images_with_age(paths):
    data = []
    labels = []
    i = 0
    for label, path in tqdm(enumerate(paths)):
        for img_path in os.listdir(path):
            image = np.array(Image.open(os.path.join(path,img_path)).convert('RGB').resize((256,256)))
            data.append(image)
            labels.append(label)
    return np.array(data), np.asarray(labels)
  

# Load all training images
print("Loading training images...")
images, ages = load_images_with_age(glob.glob(HOME_PATH + 'train_images/*'))

print(f"Loaded {len(images)} images")
print(f"Age range: {ages.min()} to {ages.max()} days")
print(f"Average age: {ages.mean():.2f} days")

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)

# Age statistics for normalization
age_mean = y_train.mean()
age_std = y_train.std()

# Normalize ages
y_train_norm = (y_train - age_mean) / age_std
y_val_norm = (y_val - age_mean) / age_std

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")

# Configuration
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 100
image_size = 72
patch_size = 6
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 8
mlp_head_units = [2048, 1024]

# Data Augmentation for age prediction
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

# Adapt normalization layer
data_augmentation.layers[0].adapt(X_train)

# MLP helper function
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

# Patches layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# Patch encoder
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# ViT Model for Age Regression
def create_vit_regressor():
    inputs = layers.Input(shape=(256, 256, 3))
    
    # Augment data
    augmented = data_augmentation(inputs)
    
    # Create patches
    patches = Patches(patch_size)(augmented)
    
    # Encode patches
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block
    for _ in range(transformer_layers):
        # Layer normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        # Create a multi-head attention layer
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        
        # Skip connection 1
        x2 = layers.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        
        # Skip connection 2
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    
    # Add MLP
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    
    # Output layer for regression (single neuron for age)
    output = layers.Dense(1, activation='linear')(features)
    
    # Create the Keras model
    model = keras.Model(inputs=inputs, outputs=output)
    return model

# Callbacks
filepath = HOME_PATH + 'paddy_models/best_vit_age_model.keras'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode='min'
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="min",
    min_delta=0.001,
    cooldown=3,
    min_lr=0
)



2.16.1
Loading training images...


10it [01:09,  6.93s/it]


Loaded 10407 images
Age range: 0 to 9 days
Average age: 5.39 days
Training samples: 8325
Validation samples: 2082


In [11]:
# Train function for regression
def train_age_model(model):
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss='mean_absolute_error',  # MAE for regression
        metrics=['mae', 'mse']
    )

    history = model.fit(
        x=X_train,
        y=y_train_norm,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(X_val, y_val_norm),
        callbacks=[checkpoint, reduce_lr],
        verbose=1
    )

    # Evaluate on validation set
    val_predictions = model.predict(X_val)
    val_predictions_original = val_predictions.flatten() * age_std + age_mean
    y_val_original = y_val

    mae = mean_absolute_error(y_val_original, val_predictions_original)
    mse = mean_squared_error(y_val_original, val_predictions_original)
    rmse = np.sqrt(mse)
    
    print(f"\nValidation MAE: {mae:.2f} days")
    print(f"Validation RMSE: {rmse:.2f} days")
    print(f"Mean age in validation: {y_val_original.mean():.2f} days")

    return history

In [12]:
# Create and train model
vit_age_model = create_vit_regressor()
vit_age_model.summary()

history = train_age_model(vit_age_model)

# Save model
vit_age_model.save_weights(HOME_PATH + 'paddy_models/vit_age.weights.h5')
vit_age_model.save(HOME_PATH + 'paddy_models/vit_age_model.keras')

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 72, 72, 3) │          7 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patches_1 (Patches) │ (None, None, 108) │          0 │ data_augmentatio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_encoder_1     │ (None, 144, 64)   │     16,192 │ patches_1[0][0]   │
│ (PatchEncoder)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 144, 64)   │        128 │ patch_encoder_1[… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 144, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 144, 64)   │          0 │ multi_head_atten… │
│                     │                   │            │ patch_encoder_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 144, 64)   │        128 │ add_16[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 144, 128)  │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 144, 128)  │          0 │ dense_21[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 144, 64)   │      8,256 │ dropout_28[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 144, 64)   │          0 │ dense_22[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_17 (Add)        │ (None, 144, 64)   │          0 │ dropout_29[0][0], │
│                     │                   │            │ add_16[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 144, 64)   │        128 │ add_17[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 144, 64)   │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_18 (Add)        │ (None, 144, 64)   │          0 │ multi_head_atten… │
│                     │                   │            │ add_17[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 144, 64)   │        128 │ add_18[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 144, 128)  │      8,320 │ layer_normalizat

 Total params: 21,657,544 (82.62 MB)

 Trainable params: 21,657,537 (82.62 MB)

 Non-trainable params: 7 (32.00 B)

Epoch 1/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 680ms/step - loss: 2.0927 - mae: 2.0927 - mse: 13.0248
Epoch 1: val_loss did not improve from 0.79609
261/261 ━━━━━━━━━━━━━━━━━━━━ 218s 723ms/step - loss: 2.0894 - mae: 2.0894 - mse: 12.9909 - val_loss: 0.8100 - val_mae: 0.8100 - val_mse: 0.9354 - learning_rate: 0.0010
Epoch 2/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - loss: 0.8264 - mae: 0.8264 - mse: 1.0468
Epoch 2: val_loss improved from 0.79609 to 0.76132, saving model to c:\Users\ThinkPad\Desktop\COSC2753_A2_MachineLearning/paddy_models/best_vit_age_model.keras
261/261 ━━━━━━━━━━━━━━━━━━━━ 184s 704ms/step - loss: 0.8264 - mae: 0.8264 - mse: 1.0468 - val_loss: 0.7613 - val_mae: 0.7613 - val_mse: 0.8795 - learning_rate: 0.0010
Epoch 3/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - loss: 0.8027 - mae: 0.8027 - mse: 1.0087
Epoch 3: val_loss improved from 0.76132 to 0.72737, saving model to c:\Users\ThinkPad\Desktop\COSC2753_A2_MachineLearning/paddy_models/best_vit_age_model.keras
261

KeyboardInterrupt: 

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MAE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# Generate predictions for test set
def load_test_images():
    test_data = []
    test_ids = []
    
    test_path = HOME_PATH + 'test_images/'
    for img_file in tqdm(os.listdir(test_path)):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(test_path, img_file)
            image = np.array(Image.open(img_path).convert('RGB').resize((256, 256)))
            test_data.append(image)
            test_ids.append(img_file.split('.')[0])
    
    return np.array(test_data), test_ids

# Load test images
print("Loading test images...")
test_images, test_ids = load_test_images()

# Make predictions
predictions = vit_age_model.predict(test_images)
predictions_original = predictions.flatten() * age_std + age_mean

# Create submission dataframe for age predictions
submission_df = pd.DataFrame({
    'image_id': test_ids,
    'age': predictions_original.astype(int).astype(str)  # Convert to string as required
})

# Save predictions
submission_df.to_csv(HOME_PATH + 'age_predictions.csv', index=False)

print(f"\nPrediction statistics:")
print(f"Min predicted age: {predictions_original.min():.2f} days")
print(f"Max predicted age: {predictions_original.max():.2f} days")
print(f"Mean predicted age: {predictions_original.mean():.2f} days")

# Display sample predictions
print("\nSample predictions:")
print(submission_df.head(10))

# Save age statistics for later use
age_stats = {
    'mean': age_mean,
    'std': age_std
}

import json
with open(HOME_PATH + 'age_stats.json', 'w') as f:
    json.dump(age_stats, f)